In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import KMeansSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import sklearn.svm as svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import time

KNN_clf = KNeighborsClassifier(n_neighbors = 3)
svm_clf =svm.SVC(kernel = 'rbf')
svm_clf_L =svm.SVC(kernel = 'linear')
lin_svc=LinearSVC(C=1)
lr_clf=LogisticRegression()
rf_clf=RandomForestClassifier(n_estimators=100, max_depth=4,random_state=0)

In [ ]:
## 데이터 불러오기

card_df=pd.read_csv('/content/drive/MyDrive/학술제 폴더/카드사기 탐지/카드사기 데이터/creditcard.csv')

In [ ]:
### 선택된 변수 넣기

df=card_df[['V17', 'V12', 'V14', 'V16', 'V10', 'V11', 'V9', 'V4', 'V7', 'V26', 'V18', 'V21', 'V1', 'V27', 'V15', 'V8', 'V20', 'V3', 'Class']]
df

,V17,V12,V14,V16,V10,V11,V9,V4,V7,V26,V18,V21,V1,V27,V15,V8,V20,V3,Class
0,0.207971,-0.617801,-0.311169,-0.470401,0.090794,-0.551600,0.363787,1.378155,0.239599,-0.189115,0.025791,-0.018307,-1.359807,0.133558,1.468177,0.098698,0.251412,2.536347,0
1,-0.114805,1.065235,-0.143772,0.463917,-0.166974,1.612727,-0.255425,0.448154,-0.078803,0.125895,-0.183361,-0.225775,1.191857,-0.008983,0.635558,0.085102,-0.069083,0.166480,0
2,1.109969,0.066084,-0.165946,-2.890083,0.207643,0.624501,-1.514654,0.379780,0.791461,-0.139097,-0.121359,0.247998,-1.358354,-0.055353,2.345865,0.247676,0.524980,1.773209,0
3,-0.684093,0.178228,-0.287924,-1.059647,-0.054952,-0.226487,-1.387024,-0.863291,0.237609,-0.221929,1.965775,-0.108300,-0.966272,0.062723,-0.631418,0.377436,-0.208038,1.792993,0
4,-0.237033,0.538196,-1.119670,-0.451449,0.753074,-0.822843,0.817739,0.403034,0.592941,0.502292,-0.038195,-0.009431,-1.158233,0.219422,0.175121,-0.270533,0.408542,1.548718,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,1.991691,2.711941,4.626942,1.107641,4.356170,-1.593105,1.914428,-2.066656,-4.918215,0.250034,0.510632,0.213454,-11.881118,0.943651,-0.924459,7.305334,1.475829,-9.834783,0
284803,-0.025693,0.915802,-0.675143,-0.711757,-0.975926,-0.150189,0.584800,-0.738589,0.024330,-0.395255,-1.221179,0.214205,-0.732789,0.068472,1.164931,0.294869,0.059616,2.035030,0
284804,0.313502,0.063119,-0.510602,0.140716,-0.484782,0.411614,0.432454,-0.557828,-0.296827,-0.087371,0.395652,0.232045,1.919565,0.004455,1.329284,0.708417,0.001396,-3.249640,0
284805,0.509928,-0.962886,0.449624,-0.608577,-0.399126,-1.933849,0.392087,0.689799,-0.686180,0.546668,1.113981,0.265245,-0.240440,0.108821,1.962563,0.679145,0.127434,0.702510,0


In [ ]:
## 학습용/테스트용 데이터 나누기
## 불균형한 데이터 이기 때문에 Stratified방식으로 나눠야 Class의 분포가 균등
## 데이터 원본 유지 위해 get_processed_df와 get_train_test_dataset 함수 만들어 진행

### train_test_split import 받기
from sklearn.model_selection import train_test_split
### 함수 생성, Time칼럼을 없애고 전처리 df 생성
def get_preprocessed_df(df=None):
  df_copy=df.copy()
  return df_copy
### 데이터 훈련, 테스트 셋으로 분류
def get_train_test_dataset(df=None):
  df_copy=get_preprocessed_df(df)
  X_features=df_copy.iloc[:,:-1]
  y_target=df_copy.iloc[:,-1]
  X_train,X_test,y_train,y_test=train_test_split(X_features,y_target,test_size=0.2,random_state=0,stratify=y_target)
  return X_train,X_test,y_train,y_test

X_train, X_test, y_train, y_test=get_train_test_dataset(card_df)

## 학습, 테스트 데이터 레이블값 비율 100 곱해서 백분율
print(y_train.value_counts()/y_train.shape[0]*100)
print(y_test.value_counts()/y_test.shape[0]*100)

0    99.827075
1     0.172925
Name: Class, dtype: float64
0    99.827955
1     0.172045
Name: Class, dtype: float64


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix, f1_score
def get_model_train_eval(model, feature_train=None, feature_test=None, target_train=None, target_test=None):
    model.fit(feature_train, target_train)
    pred=model.predict(feature_test)
    ##pred_proba=model.predict_proba(feature_test)[:,1]
    ### 혼동행렬
    confusion=confusion_matrix(target_test,pred)
    ### 정확도 
    accuracy=accuracy_score(target_test,pred)
    ### 정밀도
    precision=precision_score(target_test,pred)
    ### 재현율
    recall=recall_score(target_test,pred)
    ### f1스코어
    f1=f1_score(target_test,pred)

    X_train, X_test, y_train, y_test=get_train_test_dataset(card_df)

    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy,precision,recall,f1))

In [ ]:
### ADASYN 기법

def get_ADASYN(data, num):
  X_train, X_test, y_train, y_test=get_train_test_dataset(data)
  ads=ADASYN(random_state=42, sampling_strategy=num)
  X_train_over_ADA,y_train_over_ADA = ads.fit_resample(X_train,y_train)
  print('ADASYN 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
  print('ADASYN 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over_ADA.shape, y_train_over_ADA.shape)
  print('ADASYN 적용 후 레이블 값 분포: \n', pd.Series(y_train_over_ADA).value_counts())
  print(('ASASYN_{0}_KNN 성능').format(num))
  get_model_train_eval(KNN_clf,X_train_over_ADA,X_test,y_train_over_ADA,y_test)
  print(('ASASYN_{0}_LinearSVC 성능').format(num))
  get_model_train_eval(lin_svc,X_train_over_ADA,X_test,y_train_over_ADA,y_test)
  print(('ASASYN_{0}_SVM_Kernel=Linear 성능').format(num))
  get_model_train_eval(svm_clf_L,X_train_over_ADA,X_test,y_train_over_ADA,y_test)
  print(('ASASYN_{0}_SVM_Kernel=rbf 성능').format(num))
  get_model_train_eval(svm_clf,X_train_over_ADA,X_test,y_train_over_ADA,y_test)
  print(('ASASYN_{0}_로지스틱 회귀 성능').format(num))
  get_model_train_eval(lr_clf,X_train_over_ADA,X_test,y_train_over_ADA,y_test)
  print(('ASASYN_{0}_랜덤 포레스트 성능').format(num))
  get_model_train_eval(rf_clf,X_train_over_ADA,X_test,y_train_over_ADA,y_test)

In [ ]:
### Borderline SMOTE 기법

def get_BorderSMOTE(data, num):
  X_train, X_test, y_train, y_test=get_train_test_dataset(data)
  BSMOTE=BorderlineSMOTE(random_state=42, sampling_strategy=num)
  X_train_over_BS,y_train_over_BS = BSMOTE.fit_resample(X_train,y_train)
  print('Borderline SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
  print('Borderline SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over_BS.shape, y_train_over_BS.shape)
  print('Borderline SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over_BS).value_counts())
  print(('Borderline_SMOTE_{0}_KNN 성능').format(num))
  get_model_train_eval(KNN_clf,X_train_over_BS,X_test,y_train_over_BS,y_test)
  print(('Borderline_SMOTE_{0}_LinearSVC 성능').format(num))
  get_model_train_eval(lin_svc,X_train_over_BS,X_test,y_train_over_BS,y_test)
  print(('Borderline_SMOTE_{0}_SVM_Kernel=Linear 성능').format(num))
  get_model_train_eval(svm_clf_L,X_train_over_BS,X_test,y_train_over_BS,y_test)
  print(('Borderline_SMOTE_{0}_SVM_Kernel=rbf 성능').format(num))
  get_model_train_eval(svm_clf,X_train_over_BS,X_test,y_train_over_BS,y_test)
  print(('Borderline_SMOTE_{0}_로지스틱 회귀 성능').format(num))
  get_model_train_eval(lr_clf,X_train_over_BS,X_test,y_train_over_BS,y_test)
  print(('Borderline_SMOTE_{0}_랜덤 포레스트 성능').format(num))
  get_model_train_eval(rf_clf,X_train_over_BS,X_test,y_train_over_BS,y_test)

In [ ]:
### KMeansSMOTE 기법

def get_KMSSMOTE(data, num):
  X_train, X_test, y_train, y_test=get_train_test_dataset(data)
  KMSMOTE=KMeansSMOTE(random_state=0,cluster_balance_threshold=0.001,sampling_strategy=num)
  X_train_over_KMS,y_train_over_KMS = KMSMOTE.fit_resample(X_train,y_train)
  print('KMeansSMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
  print('KMeansSMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over_KMS.shape, y_train_over_KMS.shape)
  print('KMeansSMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over_KMS).value_counts())
  print(('KMeansSMOTE_{0}_KNN 성능').format(num))
  get_model_train_eval(KNN_clf,X_train_over_KMS,X_test,y_train_over_KMS,y_test)
  print(('KMeansSMOTE_{0}_LinearSVC 성능').format(num))
  get_model_train_eval(lin_svc,X_train_over_KMS,X_test,y_train_over_KMS,y_test)
  print(('KMeansSMOTE_{0}_SVM_Kernel=Linear 성능').format(num))
  get_model_train_eval(svm_clf_L,X_train_over_KMS,X_test,y_train_over_KMS,y_test)
  print(('KMeansSMOTE_{0}_SVM_Kernel=rbf 성능').format(num))
  get_model_train_eval(svm_clf,X_train_over_KMS,X_test,y_train_over_KMS,y_test)
  print(('KMeansSMOTE_{0}_로지스틱 회귀 성능').format(num))
  get_model_train_eval(lr_clf,X_train_over_KMS,X_test,y_train_over_KMS,y_test)
  print(('KMeansSMOTE_{0}_랜덤 포레스트 성능').format(num))
  get_model_train_eval(rf_clf,X_train_over_KMS,X_test,y_train_over_KMS,y_test)

In [ ]:
for i in (0.01, 0.02, 0.025):
  get_ADASYN(df, i)

ADASYN 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
ADASYN 적용 후 학습용 피처/레이블 데이터 세트:  (229717, 18) (229717,)
ADASYN 적용 후 레이블 값 분포: 
 0    227451
1      2266
Name: Class, dtype: int64
ASASYN_0.01_KNN 성능
정확도: 0.9988, 정밀도: 0.6160, 재현율: 0.7857, F1: 0.6906
ASASYN_0.01_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9993, 정밀도: 0.8315, 재현율: 0.7551, F1: 0.7914
ASASYN_0.01_SVM_Kernel=Linear 성능
정확도: 0.9993, 정밀도: 0.8242, 재현율: 0.7653, F1: 0.7937
ASASYN_0.01_SVM_Kernel=rbf 성능
정확도: 0.9992, 정밀도: 0.7717, 재현율: 0.7245, F1: 0.7474
ASASYN_0.01_로지스틱 회귀 성능
정확도: 0.9991, 정밀도: 0.7182, 재현율: 0.8061, F1: 0.7596
ASASYN_0.01_랜덤 포레스트 성능
정확도: 0.9993, 정밀도: 0.8152, 재현율: 0.7653, F1: 0.7895
ADASYN 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
ADASYN 적용 후 학습용 피처/레이블 데이터 세트:  (231995, 18) (231995,)
ADASYN 적용 후 레이블 값 분포: 
 0    227451
1      4544
Name: Class, dtype: int64
ASASYN_0.02_KNN 성능
정확도: 0.9984, 정밀도: 0.5302, 재현율: 0.8061, F1: 0.6397
ASASYN_0.02_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9992, 정밀도: 0.7500, 재현율: 0.7959, F1: 0.7723
ASASYN_0.02_SVM_Kernel=Linear 성능
정확도: 0.9993, 정밀도: 0.8242, 재현율: 0.7653, F1: 0.7937
ASASYN_0.02_SVM_Kernel=rbf 성능
정확도: 0.9989, 정밀도: 0.6545, 재현율: 0.7347, F1: 0.6923
ASASYN_0.02_로지스틱 회귀 성능
정확도: 0.9988, 정밀도: 0.6077, 재현율: 0.8061, F1: 0.6930
ASASYN_0.02_랜덤 포레스트 성능
정확도: 0.9992, 정밀도: 0.7624, 재현율: 0.7857, F1: 0.7739
ADASYN 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
ADASYN 적용 후 학습용 피처/레이블 데이터 세트:  (233159, 18) (233159,)
ADASYN 적용 후 레이블 값 분포: 
 0    227451
1      5708
Name: Class, dtype: int64
ASASYN_0.025_KNN 성능
정확도: 0.9984, 정밀도: 0.5163, 재현율: 0.8061, F1: 0.6295
ASASYN_0.025_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9991, 정밀도: 0.7182, 재현율: 0.8061, F1: 0.7596
ASASYN_0.025_SVM_Kernel=Linear 성능
정확도: 0.9993, 정밀도: 0.8242, 재현율: 0.7653, F1: 0.7937
ASASYN_0.025_SVM_Kernel=rbf 성능
정확도: 0.9987, 정밀도: 0.5902, 재현율: 0.7347, F1: 0.6545
ASASYN_0.025_로지스틱 회귀 성능
정확도: 0.9984, 정밀도: 0.5226, 재현율: 0.8265, F1: 0.6403
ASASYN_0.025_랜덤 포레스트 성능
정확도: 0.9992, 정밀도: 0.7573, 재현율: 0.7959, F1: 0.7761


In [ ]:
for i in (0.01, 0.02, 0.025):
  get_BorderSMOTE(df, i)

Borderline SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
Borderline SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (229725, 18) (229725,)
Borderline SMOTE 적용 후 레이블 값 분포: 
 0    227451
1      2274
Name: Class, dtype: int64
Borderline_SMOTE_0.01_KNN 성능
정확도: 0.9994, 정밀도: 0.8462, 재현율: 0.7857, F1: 0.8148
Borderline_SMOTE_0.01_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9992, 정밀도: 0.7708, 재현율: 0.7551, F1: 0.7629
Borderline_SMOTE_0.01_SVM_Kernel=Linear 성능
정확도: 0.9992, 정밀도: 0.7755, 재현율: 0.7755, F1: 0.7755
Borderline_SMOTE_0.01_SVM_Kernel=rbf 성능
정확도: 0.9993, 정밀도: 0.8554, 재현율: 0.7245, F1: 0.7845
Borderline_SMOTE_0.01_로지스틱 회귀 성능
정확도: 0.9991, 정밀도: 0.7475, 재현율: 0.7551, F1: 0.7513
Borderline_SMOTE_0.01_랜덤 포레스트 성능
정확도: 0.9993, 정밀도: 0.8085, 재현율: 0.7755, F1: 0.7917
Borderline SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
Borderline SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (232000, 18) (232000,)
Borderline SMOTE 적용 후 레이블 값 분포: 
 0    227451
1      4549
Name: Class, dtype: int64
Borderline_SMOTE_0.02_KNN 성능
정확도: 0.9994, 정밀도: 0.8280, 재현율: 0.7857, F1: 0.8063
Borderline_SMOTE_0.02_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9991, 정밀도: 0.7000, 재현율: 0.7857, F1: 0.7404
Borderline_SMOTE_0.02_SVM_Kernel=Linear 성능
정확도: 0.9991, 정밀도: 0.7156, 재현율: 0.7959, F1: 0.7536
Borderline_SMOTE_0.02_SVM_Kernel=rbf 성능
정확도: 0.9994, 정밀도: 0.8506, 재현율: 0.7551, F1: 0.8000
Borderline_SMOTE_0.02_로지스틱 회귀 성능
정확도: 0.9989, 정밀도: 0.6638, 재현율: 0.7857, F1: 0.7196
Borderline_SMOTE_0.02_랜덤 포레스트 성능
정확도: 0.9992, 정밀도: 0.7723, 재현율: 0.7959, F1: 0.7839
Borderline SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
Borderline SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (233137, 18) (233137,)
Borderline SMOTE 적용 후 레이블 값 분포: 
 0    227451
1      5686
Name: Class, dtype: int64
Borderline_SMOTE_0.025_KNN 성능
정확도: 0.9994, 정밀도: 0.8280, 재현율: 0.7857, F1: 0.8063
Borderline_SMOTE_0.025_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9989, 정밀도: 0.6610, 재현율: 0.7959, F1: 0.7222
Borderline_SMOTE_0.025_SVM_Kernel=Linear 성능
정확도: 0.9990, 정밀도: 0.6724, 재현율: 0.7959, F1: 0.7290
Borderline_SMOTE_0.025_SVM_Kernel=rbf 성능
정확도: 0.9993, 정밀도: 0.8242, 재현율: 0.7653, F1: 0.7937
Borderline_SMOTE_0.025_로지스틱 회귀 성능
정확도: 0.9989, 정밀도: 0.6471, 재현율: 0.7857, F1: 0.7097
Borderline_SMOTE_0.025_랜덤 포레스트 성능
정확도: 0.9992, 정밀도: 0.7647, 재현율: 0.7959, F1: 0.7800


In [ ]:
for i in (0.01, 0.02, 0.025):
  get_KMSSMOTE(df, i)

KMeansSMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
KMeansSMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (229728, 18) (229728,)
KMeansSMOTE 적용 후 레이블 값 분포: 
 0    227451
1      2277
Name: Class, dtype: int64
KMeansSMOTE_0.01_KNN 성능
정확도: 0.9995, 정밀도: 0.9136, 재현율: 0.7551, F1: 0.8268
KMeansSMOTE_0.01_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9993, 정밀도: 0.8571, 재현율: 0.7347, F1: 0.7912
KMeansSMOTE_0.01_SVM_Kernel=Linear 성능
정확도: 0.9993, 정밀도: 0.8242, 재현율: 0.7653, F1: 0.7937
KMeansSMOTE_0.01_SVM_Kernel=rbf 성능
정확도: 0.9994, 정밀도: 0.9231, 재현율: 0.7347, F1: 0.8182
KMeansSMOTE_0.01_로지스틱 회귀 성능
정확도: 0.9992, 정밀도: 0.8395, 재현율: 0.6939, F1: 0.7598
KMeansSMOTE_0.01_랜덤 포레스트 성능
정확도: 0.9992, 정밀도: 0.8462, 재현율: 0.6735, F1: 0.7500
KMeansSMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
KMeansSMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (232003, 18) (232003,)
KMeansSMOTE 적용 후 레이블 값 분포: 
 0    227451
1      4552
Name: Class, dtype: int64
KMeansSMOTE_0.02_KNN 성능
정확도: 0.9995, 정밀도: 0.9136, 재현율: 0.7551, F1: 0.8268
KMeansSMOTE_0.02_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9993, 정밀도: 0.8391, 재현율: 0.7449, F1: 0.7892
KMeansSMOTE_0.02_SVM_Kernel=Linear 성능
정확도: 0.9993, 정밀도: 0.8242, 재현율: 0.7653, F1: 0.7937
KMeansSMOTE_0.02_SVM_Kernel=rbf 성능
정확도: 0.9994, 정밀도: 0.9231, 재현율: 0.7347, F1: 0.8182
KMeansSMOTE_0.02_로지스틱 회귀 성능
정확도: 0.9992, 정밀도: 0.8046, 재현율: 0.7143, F1: 0.7568
KMeansSMOTE_0.02_랜덤 포레스트 성능
정확도: 0.9992, 정밀도: 0.8462, 재현율: 0.6735, F1: 0.7500
KMeansSMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (227845, 18) (227845,)
KMeansSMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (233140, 18) (233140,)
KMeansSMOTE 적용 후 레이블 값 분포: 
 0    227451
1      5689
Name: Class, dtype: int64
KMeansSMOTE_0.025_KNN 성능
정확도: 0.9994, 정밀도: 0.9024, 재현율: 0.7551, F1: 0.8222
KMeansSMOTE_0.025_LinearSVC 성능


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


정확도: 0.9993, 정밀도: 0.8295, 재현율: 0.7449, F1: 0.7849
KMeansSMOTE_0.025_SVM_Kernel=Linear 성능
정확도: 0.9993, 정밀도: 0.8242, 재현율: 0.7653, F1: 0.7937
KMeansSMOTE_0.025_SVM_Kernel=rbf 성능
정확도: 0.9994, 정밀도: 0.9114, 재현율: 0.7347, F1: 0.8136
KMeansSMOTE_0.025_로지스틱 회귀 성능
정확도: 0.9992, 정밀도: 0.7778, 재현율: 0.7143, F1: 0.7447
KMeansSMOTE_0.025_랜덤 포레스트 성능
정확도: 0.9992, 정밀도: 0.8375, 재현율: 0.6837, F1: 0.7528
